In [1]:
import pandas as pd

content = pd.read_csv("Content/content_with_categories.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

Device set to use cpu


In [3]:
import re

text = re.split(r'[.\n]', content["caption"][0])
#classifier(text)
text

['The woman that I would try',
 'Is happy with a good guy',
 "But I've been hackin' so much"]

In [4]:
sentences = text
predictions = classifier(sentences)

In [16]:
#sentences[0]
classifier("I done did everything to her. She forgave me for everything, this a forever thing. hate that I treat it like it's a whatever thing. Trust me, girl, this shit is everything to me")

[[{'label': 'anger', 'score': 0.7846493124961853},
  {'label': 'disgust', 'score': 0.14713390171527863},
  {'label': 'neutral', 'score': 0.026859601959586143},
  {'label': 'sadness', 'score': 0.024460354819893837},
  {'label': 'fear', 'score': 0.008351987227797508},
  {'label': 'joy', 'score': 0.004660976585000753},
  {'label': 'surprise', 'score': 0.003883951110765338}]]

In [10]:
predictions[0]

[{'label': 'disgust', 'score': 0.8924762010574341},
 {'label': 'anger', 'score': 0.04060265049338341},
 {'label': 'fear', 'score': 0.031226690858602524},
 {'label': 'sadness', 'score': 0.013796515762805939},
 {'label': 'joy', 'score': 0.013668936677277088},
 {'label': 'neutral', 'score': 0.006616506725549698},
 {'label': 'surprise', 'score': 0.0016124853864312172}]

In [22]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.04060265049338341},
 {'label': 'disgust', 'score': 0.8924762010574341},
 {'label': 'fear', 'score': 0.031226690858602524},
 {'label': 'joy', 'score': 0.013668936677277088},
 {'label': 'neutral', 'score': 0.006616506725549698},
 {'label': 'sadness', 'score': 0.013796515762805939},
 {'label': 'surprise', 'score': 0.0016124853864312172}]

In [23]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
id = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [24]:
for i in range(len(content["caption"])):

    id.append(content["id"][i])
    text = re.split(r'[.\n]', content["caption"][i])

    # on stocke tous les résultats de chaque phrase
    all_predictions = []
    for j in range(len(text)):
        prediction = classifier(text[j])
        all_predictions.extend(prediction)

    # maintenant on calcule le max sur toutes les phrases
    max_scores = calculate_max_emotion_scores(all_predictions)

    # puis on stocke les scores par émotion
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [25]:
emotion_scores

{'anger': [np.float64(0.0632939338684082),
  np.float64(0.35458865761756897),
  np.float64(0.009314836002886295),
  np.float64(0.19757674634456635),
  np.float64(0.03946736827492714),
  np.float64(0.4204021990299225),
  np.float64(0.6395666599273682),
  np.float64(0.02666849084198475),
  np.float64(0.36125481128692627),
  np.float64(0.06413363665342331),
  np.float64(0.8081625699996948),
  np.float64(0.6271732449531555),
  np.float64(0.3687872886657715)],
 'disgust': [np.float64(0.8924762010574341),
  np.float64(0.40567559003829956),
  np.float64(0.04705296829342842),
  np.float64(0.39774850010871887),
  np.float64(0.05671992525458336),
  np.float64(0.541950523853302),
  np.float64(0.4865266680717468),
  np.float64(0.04602104052901268),
  np.float64(0.4874332845211029),
  np.float64(0.10400672256946564),
  np.float64(0.10400672256946564),
  np.float64(0.3230978846549988),
  np.float64(0.08839710056781769)],
 'fear': [np.float64(0.033361539244651794),
  np.float64(0.03099987469613552),


In [26]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
id = []
emotion_scores = {label: [] for label in emotion_labels}

for i in range(len(content["caption"])):
    id.append(content["id"][i])
    text = re.split(r'[.\n]', content["caption"][i])

    # on stocke tous les résultats de chaque phrase
    all_predictions = []
    for j in range(len(text)):
        prediction = classifier(text[j])
        all_predictions.extend(prediction)

    # maintenant on calcule le max sur toutes les phrases
    max_scores = calculate_max_emotion_scores(all_predictions)

    # puis on stocke les scores par émotion
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [27]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["id"] = id

In [28]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,id
0,0.063294,0.892476,0.033362,0.957667,0.332168,0.295310,0.167345,a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302
1,0.354589,0.405676,0.031000,0.727903,0.789940,0.145093,0.080951,48e1a3b8-bf01-413e-86de-6fa2010d4a71
2,0.009315,0.047053,0.984090,0.158517,0.743423,0.471051,0.058835,44e2dd66-85a4-41c3-ad2b-abe4d7484258
3,0.197577,0.397749,0.026342,0.039759,0.670162,0.793203,0.033816,c4ce4c7a-0212-4749-ad65-91b69d7f661f
4,0.039467,0.056720,0.068888,0.242998,0.782880,0.026441,0.060576,0b7e90ae-628c-480a-8d26-2b5307be9ebc
5,0.420402,0.541951,0.077269,0.641668,0.275854,0.958520,0.015978,eea928f5-0ed9-402c-90b6-2dfd3c067799
6,0.639567,0.486527,0.044515,0.097134,0.620010,0.666455,0.401309,3cae97fe-1a34-4f50-9ddd-c68b16dc9ec1
7,0.026668,0.046021,0.145726,0.002703,0.569896,0.177302,0.031684,9e24db17-ddd3-4272-9115-dddd9b5bc59e
8,0.361255,0.487433,0.797973,0.419893,0.766953,0.397560,0.258139,42ba302a-c600-4d91-ae4e-d33958e16f24
9,0.064134,0.104007,0.461454,0.509706,0.927208,0.111690,0.078765,1b3a340a-4d1f-4504-a397-89a986dd076c


In [29]:
content = pd.merge(content, emotions_df, on = "id")

In [30]:
content

,id,user,text,image,caption,created_at,like,views,bookmarks,category,...,tagged_text,tagged_caption,predicted_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302,6 God,NaN,content_images/Capture.PNG,The woman that I would try\nIs happy with a go...,2025-08-26 05:19:16.592282+02,1,0,1,NaN,...,a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302: nan,a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302: The woma...,drama,0.063294,0.892476,0.033362,0.957667,0.332168,0.295310,0.167345
1,48e1a3b8-bf01-413e-86de-6fa2010d4a71,6 God,NaN,content_images/Capture_6ZjXVDF.PNG,Lucky that you stayed on\nI need someone to pu...,2025-08-26 22:39:21+02,1,0,1,NaN,...,48e1a3b8-bf01-413e-86de-6fa2010d4a71: nan,48e1a3b8-bf01-413e-86de-6fa2010d4a71: Lucky th...,drama,0.354589,0.405676,0.031000,0.727903,0.789940,0.145093,0.080951
2,44e2dd66-85a4-41c3-ad2b-abe4d7484258,6 God,NaN,content_images/https___images_kPKI5cF.genius.c...,We live in a generation of not being in love a...,2025-08-29 15:34:43.201595+02,0,0,1,NaN,...,44e2dd66-85a4-41c3-ad2b-abe4d7484258: nan,44e2dd66-85a4-41c3-ad2b-abe4d7484258: We live ...,fantasy,0.009315,0.047053,0.984090,0.158517,0.743423,0.471051,0.058835
3,c4ce4c7a-0212-4749-ad65-91b69d7f661f,6 God,NaN,content_images/https___images.genius.com_843b2...,"When a good thing goes bad, it's not the end o...",2025-08-29 15:33:35.46859+02,0,0,1,NaN,...,c4ce4c7a-0212-4749-ad65-91b69d7f661f: nan,c4ce4c7a-0212-4749-ad65-91b69d7f661f: When a g...,tragedy,0.197577,0.397749,0.026342,0.039759,0.670162,0.793203,0.033816
4,0b7e90ae-628c-480a-8d26-2b5307be9ebc,6 God,NaN,content_images/https___images.genius.com_56db9...,The biggest hit and second single from Kanye’s...,2025-08-29 15:57:36.98619+02,0,0,1,NaN,...,0b7e90ae-628c-480a-8d26-2b5307be9ebc: nan,0b7e90ae-628c-480a-8d26-2b5307be9ebc: The bigg...,historic,0.039467,0.056720,0.068888,0.242998,0.782880,0.026441,0.060576
5,eea928f5-0ed9-402c-90b6-2dfd3c067799,6 God,NaN,content_images/https___images_2nTdYSB.genius.c...,"So cry if you need to, but I can't stay to wat...",2025-08-29 15:34:01.446477+02,0,0,0,NaN,...,eea928f5-0ed9-402c-90b6-2dfd3c067799: nan,eea928f5-0ed9-402c-90b6-2dfd3c067799: So cry i...,drama,0.420402,0.541951,0.077269,0.641668,0.275854,0.958520,0.015978
6,3cae97fe-1a34-4f50-9ddd-c68b16dc9ec1,6 God,NaN,content_images/https___images.genius.com_718de...,"Mama, just killed a man\nPut a gun against his...",2025-08-29 15:55:03.343236+02,0,0,0,NaN,...,3cae97fe-1a34-4f50-9ddd-c68b16dc9ec1: nan,"3cae97fe-1a34-4f50-9ddd-c68b16dc9ec1: Mama, ju...",crime,0.639567,0.486527,0.044515,0.097134,0.620010,0.666455,0.401309
7,9e24db17-ddd3-4272-9115-dddd9b5bc59e,6 God,NaN,NaN,What do you kmow about pain,2025-08-29 15:59:02.61142+02,0,0,1,NaN,...,9e24db17-ddd3-4272-9115-dddd9b5bc59e: nan,9e24db17-ddd3-4272-9115-dddd9b5bc59e: What do ...,mystery,0.026668,0.046021,0.145726,0.002703,0.569896,0.177302,0.031684
8,42ba302a-c600-4d91-ae4e-d33958e16f24,6 God,NaN,content_images/https___images.genius.com_d309c...,The first track off Kanye West’s album 808s & ...,2025-08-29 16:00:49.881344+02,0,0,1,NaN,...,42ba302a-c600-4d91-ae4e-d33958e16f24: nan,42ba302a-c600-4d91-ae4e-d33958e16f24: The firs...,historic,0.361255,0.487433,0.797973,0.419893,0.766953,0.397560,0.258139
9,1b3a340a-4d1f-4504-a397-89a986dd076c,6 God,NaN,content_images/https___images_9obba1G.genius.c...,"Kanye’s intro to his third album, Graduation, ...",2025-08-29 16:00:14.192628+02,0,0,1,NaN,...,1b3a340a-4d1f-4504-a397-89a986dd076c: nan,1b3a340a-4d1f-4504-a397-89a986dd076c: Kanye’s ...,arts,0.064134,0.104007,0.461454,0.509706,0.927208,0.111690,0.078765


In [31]:
content.to_csv("Content/content_with_emotions.csv", index = False)